# Point Cloud Processing with Python and Open3D

## Quaranteam Hackathon - Codeacademy

# What are Point Clouds?

Point clouds are datasets that represent objects or space. These points represent the X, Y, and Z geometric coordinates of a single point on an underlying sampled surface.
They are a means of collating a large number of single spatial measurements into a dataset that can then represent a whole data.

Point clouds provide high-resolution data without the distortion sometimes present in 3D mesh models and are commonly used in industry-standard software.

<img src="Images/pic.png"> 

<img style="float: center;" src="Images/pog.png">

## Use of Point Clouds

Point clouds are used for many purposes like creating 3D CAD models for manufactured parts, for metrology and quality inspection, and for a multitude of visualization, animation, rendering and mass customization applications.

## 3D Meshes

Now, we know what point clouds are. We will look at another common term that is fairly associated with the world of 3D modelling and in this case, point clouds. That term is called 3D Mesh. 3D meshes are geometric data structures most often composed of a bunch of connected triangles that explicitly describe a surface. They are used in a wide range of applications from geospatial reconstructions to VFX, movies and video games. 

<img style="float:center;" src="Images/dino.gif">

## LOD - Levels of Details

In computer graphics, level of detail (LOD) refers to the complexity of a 3D model representation. LOD can be decreased as the model moves away from the viewer or according to other metrics such as object importance, viewpoint-relative speed or position. LOD techniques increase the efficiency of rendering by decreasing the workload on graphics pipeline stages, usually vertex transformations.

# The Code

Necessary Python libraries requires are **Numpy** and **Open3D**. 

NumPy is an open-source which offers comprehensive mathematical functions, random number generators, linear algebra routines, Fourier transforms, and more.
Open3D is an open-source library that supports rapid development of software that deals with 3D data.

In [1]:
#Imports
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#File Location
input_path="C:/Users/ashwi/Documents/Python/Projects/Point Cloud/sample_w_normals.xyz"
output_path ="C:/Users/ashwi/Documents/Python/Projects/Point Cloud/Cloud Mesh/"
point_cloud= np.loadtxt(input_path,skiprows=1)

Here we transform the point_cloud variable type from Numpy to the Open3D **o3d.geometry.PointCloud** type for further processing

In [3]:
#Point Cloud Conversion from NumPy to O3D
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6]/255)
pcd.normals = o3d.utility.Vector3dVector(point_cloud[:,6:9])

To see the imported model through Open3D, run the command.

In [4]:
o3d.visualization.draw_geometries([pcd])

## Ball Pivoting Algorithm - Method - 1

It is a type of an algorithm, where we let a "ball" which is dependent on the scale of the mesh and also larger than the average of 2 points in the mesh to find the point cloud from the particular mesh.

When the ball is rolling, it will eventually get "stuck" upon 3 points which results in the formation of a **"Seed Triangle"**. From that location, the ball rolls along the triangle edge formed from two points. The ball then settles in a new location: a new triangle is formed from two of the previous vertices and one new triangle is added to the mesh. As we continue rolling and pivoting the ball, new triangles are formed and added to the mesh. The ball continues rolling and rolling until the mesh is fully formed.

The values of the variables like Radius of the ball and Average distances between the points are based on the size and scale of the input point cloud.
Here, the smaller the Radius of the ball, more triangles, which results in more detailed mesh. Keep in mind that, the number of the traingles in a model is directly proportional to the model's depth, clarity and definition. Be aware that it will increase the computational time and CPU usage. 

<img src="Images/bpa.png"> 

In [5]:
#BPA - Variables
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

In [6]:
#Mesh Computation
bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,o3d.utility.DoubleVector([radius, radius * 2]))

In [7]:
#Mesh Decimation 
dec_mesh = o3d.geometry.TriangleMesh.simplify_quadric_decimation(bpa_mesh,100000)

After the BPA Mesh has been created, it is necessary to crop and remove all the duplicate and unprocessed points in the mesh which might yield inconsistencies throughout the model. So we use the functions provided in the Open3D library. 

In [8]:
#Mesh Consistency
dec_mesh.remove_degenerate_triangles()
dec_mesh.remove_duplicated_triangles()
dec_mesh.remove_duplicated_vertices()
dec_mesh.remove_non_manifold_edges()

TriangleMesh with 87725 points and 99981 triangles.

## Poisson Reconstruction - Method 2

The Poisson Reconstruction is a bit more mathematical as it involves computation of the implicit function along with the reconstruction of the surface based on the CGAL library's function **Delaunay Refinement**. Its approach is known as an implicit meshing method. To put it in simpler terms, we take a smooth cloth material and put it on the mesh which results in the watertight model.

<img src="Images/eros.jpg"> 

In [9]:
#Poisson Reconstruction
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]

In [10]:
#Mesh Cropping
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)

Once we clean our model, we just have to export it. 

In order to view the result, there are a lot of open-source 3D modelling software such as Blender and Meshlab. Download them, or you can either view them directly in Windows ny using the defualt 3D Model Viewer. Since, we are exporting the meshes direclty in **.ply (Polygon File Format)** format, it will be easy for us to load them and play with them in Blender. 

In [11]:
#Export
o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", dec_mesh)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)

True

# BPA vs. Poisson

Fig. 1 - BPA Mesh with the "Balls rolled"

<img src="Images/b1.png"> 

Fig. 2 - Poisson Reconstruucted Mesh

<img src="Images/b3.png"> 